In [ ]:
from langchain.agents import AgentExecutor, Tool, AgentType, initialize_agent
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
from langchain_groq import ChatGroq
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from typing import Dict, List, TypedDict, Literal, Union
import os
import json

load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [ ]:
class GraphState(TypedDict):
    """A graph state, which is a dictionary of node states."""
    requirement: str
    generated_code: str
    generated_review_comment: str
    generated_doc_string: str
    messages: List[Union[HumanMessage, SystemMessage, AIMessage]]

In [2]:
# Define the output structure for requirements
class Requirement(BaseModel):
    id: str = Field(description="Unique identifier for the requirement")
    title: str = Field(description="Short title for the requirement")
    description: str = Field(description="Detailed description of what the requirement entails")
    priority: str = Field(description="Priority level (High, Medium, Low)")
    category: str = Field(description="Category of requirement (Functional, Non-functional, UI/UX, etc.)")
    acceptance_criteria: List[str] = Field(description="List of criteria that must be met for this requirement to be considered complete")
    source: str = Field(description="Source of the requirement: 'user' for explicitly user-mentioned or 'derived' for agent-derived")


In [3]:
class RequirementsList(BaseModel):
    requirements: List[Requirement] = Field(description="List of extracted requirements")
    summary: str = Field(description="Brief summary of the requirements")

In [4]:
# Define the accumulated requirements state
class RequirementsState:
    def __init__(self):
        self.requirements = []
        self.clarifications = []
        self.prioritization_notes = []
        self.validation_notes = []
    
    def add_requirement(self, req):
        # Check if requirement with same ID already exists
        existing_ids = [r.get("id") for r in self.requirements]
        if req.get("id") in existing_ids:
            # Update existing requirement
            idx = existing_ids.index(req.get("id"))
            self.requirements[idx].update(req)
        else:
            # Add new requirement
            self.requirements.append(req)
    
    def add_requirements(self, reqs):
        for req in reqs:
            self.add_requirement(req)
    
    def add_clarification(self, clarification):
        self.clarifications.append(clarification)
    
    def add_prioritization(self, note):
        self.prioritization_notes.append(note)
    
    def add_validation(self, note):
        self.validation_notes.append(note)
    
    def to_dict(self):
        return {
            "requirements": self.requirements,
            "clarifications": self.clarifications,
            "prioritization_notes": self.prioritization_notes,
            "validation_notes": self.validation_notes
        }
    
    def get_requirements_list(self):
        # Convert to RequirementsList format
        formatted_reqs = []
        for req in self.requirements:
            # Ensure all required fields are present
            if not all(k in req for k in ["id", "title", "description", "priority", "category"]):
                continue
                
            formatted_reqs.append(Requirement(
                id=req.get("id"),
                title=req.get("title"),
                description=req.get("description"),
                priority=req.get("priority"),
                category=req.get("category"),
                acceptance_criteria=req.get("acceptance_criteria", []),
                source=req.get("source", "derived")
            ))
            
        summary = f"Collected {len(formatted_reqs)} requirements across {len(set(r.category for r in formatted_reqs))} categories."
        if self.clarifications:
            summary += f" {len(self.clarifications)} clarification points identified."
            
        return RequirementsList(
            requirements=formatted_reqs,
            summary=summary
        )

In [5]:
def extract_requirements(user_input: str) -> Dict[str, Any]:
    """
    Extract structured requirements from user input.
    This will identify explicit and implicit requirements from the text.
    
    Args:
        user_input (str): The input text containing requirement descriptions
        
    Returns:
        Dict[str, Any]: Extracted requirements with structure
    """
    return {"status": "Requirements extracted", "input": user_input}

In [6]:
# Global requirements state
reqs_state = RequirementsState()

In [7]:
# Define tool functions with state updates
def extract_requirements(user_input: str) -> str:
    """
    Extract structured requirements from user input and add to state.
    
    Args:
        user_input (str): The input text containing requirement descriptions
        
    Returns:
        str: Status of the extraction
    """
    # In a real implementation, this would use the LLM to extract requirements
    # For now, we'll simulate by adding a placeholder requirement
    reqs_state.add_requirement({
        "id": f"REQ-{len(reqs_state.requirements) + 1}",
        "title": f"Extracted Requirement {len(reqs_state.requirements) + 1}",
        "description": f"Requirement extracted from: {user_input}",
        "priority": "Medium",
        "category": "Functional",
        "acceptance_criteria": ["Criterion 1", "Criterion 2"],
        "source": "user"
    })
    print("Tool called: extract_requirements")
    return f"Extracted requirements from user input and added to state. Total requirements: {len(reqs_state.requirements)}"

def clarify_requirements(question: str) -> str:
    """
    Generate clarifying questions for ambiguous requirements.
    
    Args:
        question (str): The requirement or context that needs clarification
        
    Returns:
        str: Clarification questions generated
    """

    print("Tool called: clarify_requirements")
    clarification = f"Clarification needed: {question}"
    reqs_state.add_clarification(clarification)
    return clarification

def prioritize_requirements(requirements_json: str) -> str:
    """
    Suggest prioritization for the extracted requirements.
    
    Args:
        requirements_json (str): JSON string or description of requirements to prioritize
        
    Returns:
        str: Prioritization results
    """

    print("Tool called: prioritize_requirements")
    note = f"Prioritization note: {requirements_json}"
    reqs_state.add_prioritization(note)
    return note

def validate_requirements(requirements_json: str) -> str:
    """
    Check requirements for consistency, completeness, and clarity.
    
    Args:
        requirements_json (str): JSON string or description of requirements to validate
        
    Returns:
        str: Validation results
    """

    print("Tool called: extract_requirements")
    validation = f"Validation: {requirements_json}"
    reqs_state.add_validation(validation)
    return validation

def get_final_requirements() -> str:
    """
    Get the final structured requirements list.
    
    Returns:
        str: JSON string of the structured requirements
    """

    print("Tool called: get_final_requirements")
    reqs_list = reqs_state.get_requirements_list()
    return json.dumps(reqs_list.dict(), indent=2)


In [8]:
# Create the requirements agent using Groq
def create_requirements_agent(groq_api_key=None, model_name="llama3-8b-8192", temperature=0.2):
    # Reset the global state
    global reqs_state
    reqs_state = RequirementsState()
    
    # Set API key if provided
    if groq_api_key:
        os.environ["GROQ_API_KEY"] = groq_api_key
    
    # Initialize the Groq language model
    llm = ChatGroq(
        model_name=model_name,
        temperature=temperature
    )
    
    # Define the tools
    tools = [
        Tool(
            name="ExtractRequirements",
            func=extract_requirements,
            description="Extract structured requirements from user input. Use this to identify and structure requirements from text."
        ),
        Tool(
            name="ClarifyRequirements",
            func=clarify_requirements,
            description="Generate questions to clarify ambiguous requirements. Use this when requirements are unclear or incomplete."
        ),
        Tool(
            name="PrioritizeRequirements",
            func=prioritize_requirements,
            description="Suggest prioritization for requirements. Use this to rank requirements by importance and impact."
        ),
        Tool(
            name="ValidateRequirements",
            func=validate_requirements,
            description="Check requirements for consistency and completeness. Use this to verify requirements quality."
        ),
        Tool(
            name="GetFinalRequirements",
            func=get_final_requirements,
            description="Get the final structured requirements list. Use this at the end to retrieve the complete requirements."
        )
    ]
    
    # Create memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # System message to ensure the agent follows the process and uses the final tool
    system_message = """You are a Requirements Analysis Agent. Your job is to extract, clarify, prioritize, and validate software requirements from user inputs.

IMPORTANT: Always follow this process:
1. Use ExtractRequirements to identify and structure requirements from the user's input
2. Use ClarifyRequirements if anything is ambiguous
3. Use PrioritizeRequirements to suggest priority levels
4. Use ValidateRequirements to check quality and completeness
5. At the end of your analysis, ALWAYS use GetFinalRequirements to retrieve the final structured output

When you get the final requirements, present them in a well-formatted way to the user. Make sure all user-provided requirements are captured and structured properly."""
    
    # Create the agent executor using the zero-shot-react-description approach
    agent_executor = initialize_agent(
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        tools=tools,
        llm=llm,
        verbose=True,
        max_iterations=7,
        memory=memory,
        handle_parsing_errors=True,
        early_stopping_method="generate",
        agent_kwargs={"system_message": system_message}
    )
    
    return agent_executor

In [9]:
# Example of how to use with Groq
def requirements_agent_example_groq(groq_api_key=None):
    # Creating a standalone example
    req_agent = create_requirements_agent(groq_api_key)
    
    # Example user input
    user_input = """
    We need a user authentication system that allows users to register, login, and reset their 
    passwords. Users should be able to use social login options like Google and Facebook.
    The system needs to be secure and should use two-factor authentication for sensitive operations.
    The login page should be responsive and work well on mobile devices.
    """
    
    # Process the requirements
    result = req_agent.invoke({"input": user_input})
    
    # Get the final structured requirements
    final_requirements = reqs_state.get_requirements_list()
    
    # Print the final requirements
    print("\n\nFINAL STRUCTURED REQUIREMENTS:")
    print(json.dumps(final_requirements.dict(), indent=2))
    
    return final_requirements

In [10]:
user_input = """ DreamBigin - Web Application Requirements
A social media platform for artists to collaborate and showcase their work.

1. User Management
1.1 Registration & Login
Users should be able to register and log in using OAuth authentication (Google, Twitter, Facebook, etc.).
Users should have the option to register with email and password as an alternative.
Implement two-factor authentication (2FA) for enhanced security.
Implement role-based access control for different types of users (Artists, Businesses, Admins).
1.2 User Profile
Users should be able to create and manage their profile with:
Profile Picture
Bio / Short Description
List of Skills (e.g., Acting, Guitar, Directing)
Social Media Links (Instagram, YouTube, TikTok, LinkedIn, Twitter, etc.)
Portfolio Section (Showcase previous work with images/videos)
1.3 Collaboration Requests
Users should be able to create collaboration projects specifying:
Title
Project Description
Required Skills (e.g., Cinematographer, Scriptwriter, Vocalist)
Project Type (Short Film, Music Band, Art Collaboration, etc.)
Collaboration Location (Remote / Onsite)
Deadline (if applicable)
Compensation (if any)
Interested artists can apply to collaborate or directly message the project owner.
2. Business Enrollment & Advertisements
2.1 Business Accounts
Businesses (e.g., Camera Rentals, Film Institutes) should be able to:
Register as a Business Account.
List services (e.g., Equipment Rental, Online/Offline Courses).
Promote offers and discounts on rentals or training programs.
Receive inquiries from artists through direct messaging.
2.2 Business Promotions
Business accounts should have an option to boost their services (paid promotion).
Integration with Google Ads / Facebook Ads API for external promotions.
Businesses should be able to post job openings (e.g., “Seeking Assistant Director for an upcoming project”).
3. Content Sharing & Engagement
3.1 Content Creation
Users should be able to write articles and blogs (like Medium).
Users can post external links (e.g., YouTube, Instagram, SoundCloud).
Users can upload images & video snippets to showcase their work.
3.2 Engagement Features
Like, Comment, and Share features for posts.
Follow / Unfollow feature to subscribe to other users’ content.
Trending Content Section based on user engagement.
Hashtags & Categories for easy discoverability.
Users should be able to embed multimedia (videos, audio, GIFs).
4. Messaging & Notifications
4.1 Direct Messaging
Real-time chat for users to discuss collaborations.
Businesses can communicate with artists about services.
Group Chats for multi-person collaboration discussions.
4.2 Notifications
In-app notifications for new collaboration requests, messages, likes, comments, etc.
Email & Push Notifications (configurable in settings).
Weekly summary emails about trending posts and collaboration opportunities.
5. Discoverability & Search
5.1 Search & Filters
Search for Users, Projects, Businesses, and Content.
Filters by Skills, Location, Availability, Experience Level.
AI-powered recommendations based on user interests and engagement.
5.2 Featured Artists & Businesses
A curated list of top-rated artists based on collaborations and community engagement.
Business Spotlights showcasing top rental providers and training institutes.
6. Monetization & Payments
6.1 Premium Features
Subscription-based premium profiles for enhanced visibility.
Paid promoted collaboration requests to reach more artists.
6.2 Payment Gateway
Integration with Stripe / PayPal for in-app payments.
Artists can donate to other artists or fund a project.
Businesses can accept payments for courses & rentals.
7. Admin Panel
User & Content Moderation (Report & Ban system for abusive content).
Collaboration Request Monitoring (Prevent spam/fraudulent requests).
Analytics Dashboard (Track engagement, active users, trending skills).
8. Technology Stack (Suggested)
Frontend: React / Angular with TypeScript & SCSS.
Backend: .NET Core (C#) or Node.js (Express) with REST APIs.
Database: PostgreSQL / MySQL for structured data.
Authentication: OAuth, Firebase Auth, or Identity Server.
Messaging & Notifications: WebSockets for real-time chat, Firebase for push notifications.
File Storage: AWS S3 / Azure Blob Storage for multimedia.
AI Integration: AI-powered recommendations for matching artists & projects.
Additional Enhancements
✅ AI-based Profile Recommendations – Suggests potential collaborations based on skills.
✅ NFT Art Marketplace (Future Scope) – Artists can sell digital artwork as NFTs.
✅ Live Streaming Events – Artists and businesses can host live Q&A sessions.
✅ Mobile App (Future Scope) – Extend functionality to iOS/Android apps. """

In [11]:
# process_requirements(user_input)
# Example of how to use with Groq
def requirements_agent_example_groq(groq_api_key=None):
    # Creating a standalone example
    req_agent = create_requirements_agent(groq_api_key)
    
    # Example user input
    user_input = """
    We need a user authentication system that allows users to register, login, and reset their 
    passwords. Users should be able to use social login options like Google and Facebook.
    The system needs to be secure and should use two-factor authentication for sensitive operations.
    The login page should be responsive and work well on mobile devices.
    """
    
    # Process the requirements
    result = req_agent.invoke({"input": user_input})
    print(result)
    
    return result


In [12]:
result = requirements_agent_example_groq()

C:\Users\Deepak\AppData\Local\Temp\ipykernel_7020\3409952290.py:47: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Deepak\AppData\Local\Temp\ipykernel_7020\3409952290.py:62: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_execut



> Entering new AgentExecutor chain...
Thought: The input question describes a user authentication system with specific requirements. To structure these requirements, I need to extract the key points and identify any ambiguities.

Action: ExtractRequirements
Action Input: "We need a user authentication system that allows users to register, login, and reset their passwords. Users should be able to use social login options like Google and Facebook. The system needs to be secure and should use two-factor authentication for sensitive operations. The login page should be responsive and work well on mobile devices."Tool called: extract_requirements

Observation: Extracted requirements from user input and added to state. Total requirements: 1
Thought:Thought: The extracted requirements seem to be a good starting point, but I notice that the requirement for the login page to be responsive and work well on mobile devices might be specific to the user interface, which is not directly related to

In [15]:
result

{'input': '\n    We need a user authentication system that allows users to register, login, and reset their \n    passwords. Users should be able to use social login options like Google and Facebook.\n    The system needs to be secure and should use two-factor authentication for sensitive operations.\n    The login page should be responsive and work well on mobile devices.\n    ',
 'chat_history': [HumanMessage(content='\n    We need a user authentication system that allows users to register, login, and reset their \n    passwords. Users should be able to use social login options like Google and Facebook.\n    The system needs to be secure and should use two-factor authentication for sensitive operations.\n    The login page should be responsive and work well on mobile devices.\n    ', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Here's the continuation:\n\nThought: Since the clarification needed is to understand what specific design and functionality elements need

In [18]:
for k,v in result.items():
    print(k)
    print(v)
    print("\n\n")

input

    We need a user authentication system that allows users to register, login, and reset their 
    passwords. Users should be able to use social login options like Google and Facebook.
    The system needs to be secure and should use two-factor authentication for sensitive operations.
    The login page should be responsive and work well on mobile devices.
    



chat_history
[HumanMessage(content='\n    We need a user authentication system that allows users to register, login, and reset their \n    passwords. Users should be able to use social login options like Google and Facebook.\n    The system needs to be secure and should use two-factor authentication for sensitive operations.\n    The login page should be responsive and work well on mobile devices.\n    ', additional_kwargs={}, response_metadata={}), AIMessage(content="Here's the continuation:\n\nThought: Since the clarification needed is to understand what specific design and functionality elements need to be included

In [14]:
# Example implementation of a LangGraph compatible node
class RequirementsNode:
    def __init__(self, groq_api_key=None, model_name="llama3-8b-8192"):
        self.agent = create_requirements_agent(groq_api_key, model_name)
    
    def __call__(self, state):
        # Extract input from state
        user_input = state.get("user_input", "")
        
        # Process the requirements
        result = self.agent.invoke({"input": user_input})
        
        # Get the structured requirements
        requirements_list = reqs_state.get_requirements_list()
        
        # Update the state with processed requirements
        updated_state = state.copy()
        updated_state["requirements"] = requirements_list.dict()
        updated_state["requirements_raw"] = result
        
        return updated_state


In [19]:
def reverse_new_list(arr):
    return arr[::-1] 

In [20]:
arr =[5,4,3,2,1]
rev = reverse_new_list(arr)
rev

[1, 2, 3, 4, 5]

In [24]:
arr[::-1]

[1, 2, 3, 4, 5]

In [70]:
arr1 = [1, 4, 6]  
arr2 = [2, 3, 5, 8, 10]

In [73]:
def merge_sorted_array(arr1,arr2):
    i, j = 0, 0
    merged_arr = []

    while i < len(arr1) and j < len(arr2):
        print(f"i={i}, j={j}")
        if arr1[i] < arr2[j]:
            merged_arr.append(arr1[i])
            i += 1
        else:
            merged_arr.append(arr2[j])
            j+=1
        
    while i < len(arr1):
        merged_arr.append(arr1[i])
        i += 1

    while j < len(arr2):
        merged_arr.append(arr2[j])
        j+=1
    
    return merged_arr


In [74]:
arr3 = merge_sorted_array(arr1, arr2)
arr3

i=0, j=0
i=1, j=0
i=1, j=1
i=1, j=2
i=2, j=2
i=2, j=3


[1, 2, 3, 4, 5, 6, 8, 10]

In [83]:
arr = [0,1,0,3,12]

def move_zeroes(arr):
    beg, end = 0, len(arr) - 1
    while beg < end:
        print(f"beg={beg} and element is {arr[beg]}, end={end} and element is {arr[end]}")
        if arr[beg] == 0:
            arr.pop(beg)
            arr.append(0)
            # arr[end] = arr[beg]
            beg += 1
        else:
            end -= 1
    return arr

In [84]:
rev_arr = move_zeroes(arr)
rev_arr

beg=0 and element is 0, end=4 and element is 12
beg=1 and element is 0, end=4 and element is 0
beg=2 and element is 12, end=4 and element is 0
beg=2 and element is 12, end=3 and element is 0


[1, 3, 12, 0, 0]

In [89]:
def move_zeroes(arr):
    beg = 0  # Pointer for non-zero elements

    # Move all non-zero elements to the front
    for i in range(len(arr)):
        print(f"i={i} and element is {arr[i]}, beg={beg} and element is {arr[beg]}")
        print(f"state of arr is {arr}")
        if arr[i] != 0:
            print("swapping")
            arr[beg], arr[i] = arr[i], arr[beg]  # Swap
            beg += 1  # Move pointer for next non-zero

    return arr  # The zeroes are automatically at the end

In [91]:
print(move_zeroes([0, 1, 3, 0, 12]))  # Output: [1, 3, 12, 0, 0]

i=0 and element is 0, beg=0 and element is 0
state of arr is [0, 1, 3, 0, 12]
i=1 and element is 1, beg=0 and element is 0
state of arr is [0, 1, 3, 0, 12]
swapping
i=2 and element is 3, beg=1 and element is 0
state of arr is [1, 0, 3, 0, 12]
swapping
i=3 and element is 0, beg=2 and element is 0
state of arr is [1, 3, 0, 0, 12]
i=4 and element is 12, beg=2 and element is 0
state of arr is [1, 3, 0, 0, 12]
swapping
[1, 3, 12, 0, 0]


In [94]:
def two_sum_hashing(arr, target):
    seen = set()  # Hash set to store numbers
    for num in arr:
        print(f"seen is = {seen}, target-num is {target-num}")
        if target - num in seen:
            return [target - num, num]
        seen.add(num)
    return []

print(two_sum_hashing([7, 2, 5, 3, 1], 8))  # Output: [5, 3]


seen is = set(), target-num is 1
seen is = {7}, target-num is 6
seen is = {2, 7}, target-num is 3
seen is = {2, 5, 7}, target-num is 5
[5, 3]


In [ ]:
# Problem 1: Maximum Sum of k Consecutive Elements (Fixed Window)
def max_sum_subarray(arr, k):
    max_sum = float('-inf')
    window_sum = sum(arr[:k])  # Sum of first window

    for i in range(len(arr) - k):
        print(f"i={i}, i+k = {i+k}, window_sum={window_sum}, max_sum={max_sum},arr[i]={arr[i]}, arr[i+k]={arr[i+k]}")
        window_sum = window_sum - arr[i] + arr[i + k]  # Slide the window
        max_sum = max(max_sum, window_sum)

    return max_sum

print(max_sum_subarray([2, 1, 5, 1, 3, 2, 2], 3))  # Output: 9

i=0, i+k = 3, window_sum=8, max_sum=-inf,arr[i]=2, arr[i+k]=1
i=1, i+k = 4, window_sum=7, max_sum=7,arr[i]=1, arr[i+k]=3
i=2, i+k = 5, window_sum=9, max_sum=9,arr[i]=5, arr[i+k]=2
i=3, i+k = 6, window_sum=6, max_sum=9,arr[i]=1, arr[i+k]=2
9


In [121]:
# Problem 2: Smallest Subarray Sum ≥ Target (Variable Window)
def min_subarray_length(arr, target):
    left = 0
    min_length = float('inf')
    current_sum = 0

    for right in range(len(arr)):  # Expand window
        print(f"left={left}, right={right}, current_sum={current_sum}, min_length={min_length}, arr[right]={arr[right]}")
        current_sum += arr[right]

        while current_sum >= target:  # Shrink window when condition is met
            print("In while loop")
            print(f"left={left}, right={right}, current_sum={current_sum}, min_length={min_length}, arr[left]={arr[left]}, right - left + 1={right - left + 1}")
            min_length = min(min_length, right - left + 1)
            print(f"min_length={min_length}")
            current_sum -= arr[left]
            left += 1  # Move left pointer

    return min_length if min_length != float('inf') else 0,arr[left-1:right-left+1]

print(min_subarray_length([2, 3, 1, 2, 1], 5))  

left=0, right=0, current_sum=0, min_length=inf, arr[right]=2
left=0, right=1, current_sum=2, min_length=inf, arr[right]=3
In while loop
left=0, right=1, current_sum=5, min_length=inf, arr[left]=2, right - left + 1=2
min_length=2
left=1, right=2, current_sum=3, min_length=2, arr[right]=1
left=1, right=3, current_sum=4, min_length=2, arr[right]=2
In while loop
left=1, right=3, current_sum=6, min_length=2, arr[left]=3, right - left + 1=3
min_length=2
left=2, right=4, current_sum=3, min_length=2, arr[right]=1
(2, [3, 1])


In [150]:
def longest_subarray_with_sum(arr, target):
    left = 0
    max_length = 0
    current_sum = 0
    best_subarray = []  # Store the best subarray

    for right in range(len(arr)):  # Expand window
        # best_subarray = [] 
        print("In for loop")
        print(f"left={left}, right={right}, current_sum={current_sum}, max_length={max_length}, arr[right]={arr[right]}")
        current_sum += arr[right]

        while current_sum >= target:  # Shrink window if sum exceeds target
            print("In while loop")
            print(f"left={left}, right={right}, current_sum={current_sum}, max_length={max_length}, arr[left]={arr[left]}")
            # max_length = max(max_length, right - left + 1)
            current_sum -= arr[left]
            left += 1
            print("\n")

        # Check if we found a longer subarray
        if right - left + 1 >= max_length:
            print("checking if we found a longer subarray")
            print(f"right - left + 1={right - left +1},left={left}, right+1={right+1} ")
            # max_length = max(max_length, right - left + 1)
            max_length = right - left + 1
            best_subarray = arr[left:right + 1]  # Store the subarray
            print(f"max_length={max_length}, best_subarray={best_subarray}")
        print("\n\n")

    return max_length, best_subarray

print("\n")
# Example Test Cases
# print(longest_subarray_with_sum([1, 2, 3, 4, 5], 8))  # Output: (3, [1, 2, 3])
print(longest_subarray_with_sum([2, 3, 1, 2, 4, 3], 7))  # Output: (3, [3, 1, 2])
# print(longest_subarray_with_sum([2, 1, 5, 1, 3, 2], 7))  # Output: (2, [5, 1])



In for loop
left=0, right=0, current_sum=0, max_length=0, arr[right]=2
checking if we found a longer subarray
right - left + 1=1,left=0, right+1=1 
max_length=1, best_subarray=[2]



In for loop
left=0, right=1, current_sum=2, max_length=1, arr[right]=3
checking if we found a longer subarray
right - left + 1=2,left=0, right+1=2 
max_length=2, best_subarray=[2, 3]



In for loop
left=0, right=2, current_sum=5, max_length=2, arr[right]=1
checking if we found a longer subarray
right - left + 1=3,left=0, right+1=3 
max_length=3, best_subarray=[2, 3, 1]



In for loop
left=0, right=3, current_sum=6, max_length=3, arr[right]=2
In while loop
left=0, right=3, current_sum=8, max_length=3, arr[left]=2


checking if we found a longer subarray
right - left + 1=3,left=1, right+1=4 
max_length=3, best_subarray=[3, 1, 2]



In for loop
left=1, right=4, current_sum=6, max_length=3, arr[right]=4
In while loop
left=1, right=4, current_sum=10, max_length=3, arr[left]=3


In while loop
left=2, right=4, 

In [12]:
def longest_unique_substring(s):
    char_set = set()  # HashSet to track unique characters
    left = 0  # Left pointer of window
    max_length = 0  # Maximum substring length

    for right in range(len(s)):  # Expand window
        print("In for loop")
        print(f"left={left}, right={right}, max_length={max_length}, s[right]={s[right]}")
        print(f"char_set={char_set}")
        while s[right] in char_set:  # If duplicate found, shrink window
            print("In while loop shrnking window")
            print(f"checked for {s[right]} in char_set {char_set},left={left}, right={right},s[left]={s[left]}, s[right]={s[right]}")
            print(f"removing {s[left]}")
            char_set.remove(s[left])  # Remove leftmost character
            
            left += 1  # Move left pointer
            print(f"char_set={char_set},left={left} at end of while loop")
            print("\n")

        char_set.add(s[right])  # Add new character to set
        max_length = max(max_length, right - left + 1)  # Update max length
        print(f"char_set={char_set}, max_length={max_length} at end of for loop")
        print("\n")

    return max_length,char_set

# Test Cases
# print(longest_unique_substring("abba"))  # Output: 2
print(longest_unique_substring("abcabcbb"))  # Output: 3
# print(longest_unique_substring("bbbbb"))     # Output: 1
# print(longest_unique_substring("pwwkew"))    # Output: 3
# print(longest_unique_substring(""))          # Output: 0
# print(longest_unique_substring("abcdef"))    # Output: 6


In for loop
left=0, right=0, max_length=0, s[right]=a
char_set=set()
char_set={'a'}, max_length=1 at end of for loop


In for loop
left=0, right=1, max_length=1, s[right]=b
char_set={'a'}
char_set={'b', 'a'}, max_length=2 at end of for loop


In for loop
left=0, right=2, max_length=2, s[right]=c
char_set={'b', 'a'}
char_set={'c', 'b', 'a'}, max_length=3 at end of for loop


In for loop
left=0, right=3, max_length=3, s[right]=a
char_set={'c', 'b', 'a'}
In while loop shrnking window
checked for a in char_set {'c', 'b', 'a'},left=0, right=3,s[left]=a, s[right]=a
removing a
char_set={'c', 'b'},left=1 at end of while loop


char_set={'c', 'b', 'a'}, max_length=3 at end of for loop


In for loop
left=1, right=4, max_length=3, s[right]=b
char_set={'c', 'b', 'a'}
In while loop shrnking window
checked for b in char_set {'c', 'b', 'a'},left=1, right=4,s[left]=b, s[right]=b
removing b
char_set={'c', 'a'},left=2 at end of while loop


char_set={'c', 'b', 'a'}, max_length=3 at end of for loop


In 

In [18]:
# the longest contiguous subarray whose sum is ≤ target.
def longest_subarray_with_sum(arr, target):
    left = 0
    current_sum = 0
    max_length = 0
    best_subarray = []

    for right in range(len(arr)):  # Expand window
        current_sum += arr[right]

        # Shrink window if sum exceeds target
        while current_sum > target:
            current_sum -= arr[left]
            left += 1

        # Update longest subarray when we find a new max length
        if right - left + 1 > max_length:
            max_length = right - left + 1
            best_subarray = arr[left:right + 1]

    return max_length, best_subarray

# Test Cases
print(longest_subarray_with_sum([1, 2, 3, 4, 5], 8))  # Output: (3, [1, 2, 3] or [2, 3, 4])
print(longest_subarray_with_sum([3, 1, 2, 1, 0], 5))  # Output: (3, [1, 2, 1])
print(longest_subarray_with_sum([2, 3, 1, 2, 4, 3], 7))  # Output: (3, [2, 3, 1] or [3, 1, 2])
print(longest_subarray_with_sum([5, 1, 2, 3, 4], 10))  # Output: (4, [1, 2, 3, 4])


(3, [1, 2, 3])
(4, [1, 2, 1, 0])
(3, [2, 3, 1])
(4, [1, 2, 3, 4])


In [19]:
#Selection Sort 
def findSmallest(arr):
    smallest = arr[0] #Stores the smallest value
    smallest_index = 0 #Stores the index of the smallest value
    for i in range(1, len(arr)):
        if arr[i] < smallest:
            smallest = arr[i]
            smallest_index = i
    return smallest_index

#Now you can use this function to write selection sort:
def selectionSort(arr): #Sorts an array
    newArr = []
    for i in range(len(arr)):
        smallest = findSmallest(arr)
        newArr.append(arr.pop(smallest))
    return newArr

print(selectionSort([5, 3, 6, 2, 10]))

[2, 3, 5, 6, 10]


In [6]:


def sum(list):

    if list == []:
        return 0
    return list[0] + sum(list[1:])

sum([2,4,6])

12

In [40]:
def bin_search(arr, search_num):
    mid = int(len(arr)//2.0)
    # print(mid,arr[mid])
    if mid == 0 and arr[0]!=search_num:
        return -1
    if search_num == arr[mid]:
        return search_num
    # elif arr[mid] > search_num:
    elif search_num < arr[mid]:
       return bin_search(arr[0:mid],search_num)
    else:
        return bin_search(arr[mid:],search_num)
    
   
check = bin_search([2,4,6,7,10,12], 14)
check

-1